# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
import random
from collections import Counter
from pyqrack import QrackSimulator

# Range is up to 2 ** input_size - 1
input_size = 60

# Arbitrary value with range:
hidden_bits = 2973

# Here's the catch: this one will fail, with "default optimal" PyQrack, or in general
# hidden_bits = 2^60 - 10923

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

oracle_swap_tuples = []
for i in range(10):
    i = random.randint(input_size, input_size * 2)
    j = random.randint(input_size, input_size * 2)
    if i != j:
        oracle_swap_tuples.append((i, j))

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    # for i in oracle_swap_tuples:
    #      sim.swap(i[0], i[1])

This is the algorithm itself. (A classical simulator of quantum computers can run the "unitary preamble" exactly once and then secondarily "sample" the measurement distribution of the end state.)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
results = Counter(sim.measure_shots(oracle_qubits, 10000))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


...The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

2973 . 4105997242 = 0 (mod 2)
2973 . 238605814 = 0 (mod 2)
2973 . 2613538804 = 0 (mod 2)
2973 . 1317218835 = 0 (mod 2)
2973 . 1164871362 = 0 (mod 2)
2973 . 172502961 = 0 (mod 2)
2973 . 3750820826 = 0 (mod 2)
2973 . 3291282420 = 0 (mod 2)
2973 . 1936485773 = 0 (mod 2)
2973 . 3227371402 = 0 (mod 2)
2973 . 1973947691 = 0 (mod 2)
2973 . 1036849922 = 0 (mod 2)
2973 . 593721295 = 0 (mod 2)
2973 . 2620330420 = 0 (mod 2)
2973 . 1941284639 = 0 (mod 2)
2973 . 4153532961 = 0 (mod 2)
2973 . 3526240391 = 0 (mod 2)
2973 . 1125482951 = 0 (mod 2)
2973 . 4200043402 = 0 (mod 2)
2973 . 2743405628 = 0 (mod 2)
2973 . 3941035658 = 0 (mod 2)
2973 . 829461066 = 0 (mod 2)
2973 . 3680463329 = 0 (mod 2)
2973 . 1290429099 = 0 (mod 2)
2973 . 1683487685 = 0 (mod 2)
2973 . 2837064099 = 0 (mod 2)
2973 . 3633570929 = 0 (mod 2)
2973 . 1890436646 = 0 (mod 2)
2973 . 1394713505 = 0 (mod 2)
2973 . 3690182368 = 0 (mod 2)
2973 . 4040822375 = 0 (mod 2)
2973 . 1246846359 = 0 (mod 2)
2973 . 838273341 = 0 (mod 2)
2973 . 69903008